In [1]:
import pandas as pd
import numpy as np
import datetime
import csv
import time
from tqdm import trange
from xgboost import XGBClassifier

In [2]:
import struct
print(struct.calcsize("P") * 8)

64


In [3]:
#這部分是creatine病人
source=pd.read_csv('/Users/owen/Documents/【專案助理】/資料/呼吸重症資料/data.csv')
source.shape[0]

7104

In [4]:
source = source.drop(labels=["Unnamed: 0"], axis="columns")
source.head()

,GUID_2,GUID_1,blood culture in ICU,性別,此次入院年齡,本院初次住院就診日,本院初次急診就診日,本院初次門診就診日,此次住院入院日,此次入院最後出院日,...,Diabetes without chronic complication,Diabetes with chronic complication,Hemiplegia or paraplegia,Renal disease,"Any malignancy, including lymphoma and leukaemia, except malignant neoplasm of skin",Moderate or severe liver disease,Metastatic solid tumour,AIDS/HIV,CCI,出院科別
0,0002e0909d70d7138dd33940ae71f4bb84c95ff8b3707a...,9ba8170486ff7bd345773f71823611bcf3efba85854c0f...,N,F,72,2015/5/15 16:49,2015/5/15 13:34,2015/5/25,2015/5/15 16:49,2015/6/5 10:20,...,1,0,0,0,0,0,0,0,3,NEUR
1,0017d37011c2ae9e484d45bc86aa402391a7fc059c86fc...,9f4e06fbf790992a71065e0312e42679280f30f9d25432...,Y,M,66,2010/3/22 9:38,2010/5/13 15:24,2010/3/10,2017/12/28 21:11,2018/7/19 11:21,...,0,0,0,1,1,1,0,0,4,GS
2,001c72b33dca120ccf0ced078c0a740b8188e7ccfea828...,120e61975fd7669695985576d334ef4e3ad3647e8606ae...,Y,M,69,2017/9/10 10:12,2017/9/9 10:29,2017/9/29,2017/12/22 11:27,2018/1/24 16:00,...,0,0,0,0,1,0,1,0,2,ORTH
3,0024253c798d1e16d34c325c0784141e0e978090e723a2...,791033ef38a659f0cc8a7ba6fc6d90ec6393a6f5f518ac...,Y,M,64,2016/12/8 11:27,2016/12/8 11:03,NaN,2016/12/8 11:27,2017/2/2 22:32,...,0,0,0,0,0,0,0,0,0,NS
4,00307b46c5b0c530025f17c45103b42b6dbcefc6865b78...,853550c39c93a40cf9b8cdd63c4eba362ed5ae4d1d0ed7...,Y,M,84,2003/2/18 13:36,2017/1/26 22:59,NaN,2017/1/27 16:17,2017/2/11 5:00,...,0,0,0,1,0,0,0,0,2,NEPH


In [5]:
for col in source.columns:
    print(col)

GUID_2
GUID_1
blood culture in ICU
性別
此次入院年齡
本院初次住院就診日
本院初次急診就診日
本院初次門診就診日
此次住院入院日
此次入院最後出院日
此次入院急診日
此次ICU轉入日
此次ICU轉出日
病床
此次入院進CU頻次
出院30日
die(30)
出院90日
die(90)
出院180日
die(180)
出院365日
die(365)
出院狀態
die
健保死亡紀錄
D1_StartTime
D1_EndTime
D2_StartTime
D2_EndTime
D3_StartTime
D3_EndTime
D4_StartTime
D4_EndTime
D5_StartTime
D5_EndTime
D6_StartTime
D6_EndTime
D7_StartTime
D7_EndTime
D1_FIO2
D2_FIO2
D3_FIO2
D4_FIO2
D5_FIO2
D6_FIO2
D7_FIO2
D1_PEEPCPAP
D2_PEEPCPAP
D3_PEEPCPAP
D4_PEEPCPAP
D5_PEEPCPAP
D6_PEEPCPAP
D7_PEEPCPAP
D1_PAW
D2_PAW
D3_PAW
D4_PAW
D5_PAW
D6_PAW
D7_PAW
D1_MAPS
D2_MAPS
D3_MAPS
D4_MAPS
D5_MAPS
D6_MAPS
D7_MAPS
D1_TOTRR
D2_TOTRR
D3_TOTRR
D4_TOTRR
D5_TOTRR
D6_TOTRR
D7_TOTRR
D1_VTEXH
D2_VTEXH
D3_VTEXH
D4_VTEXH
D5_VTEXH
D6_VTEXH
D7_VTEXH
D1_MVEXH
D2_MVEXH
D3_MVEXH
D4_MVEXH
D5_MVEXH
D6_MVEXH
D7_MVEXH
D1_DDP
D2_DDP
D3_DDP
D4_DDP
D5_DDP
D6_DDP
D7_DDP
D2-D1_FIO2
D3-D2_FIO2
D4-D3_FIO2
D5-D4_FIO2
D6-D5_FIO2
D7-D6_FIO2
D2-D1_PEEPCPAP
D3-D2_PEEPCPAP
D4-D3_PEEPCPAP
D5-D4_PEEPCPAP
D6-D5_PEEPCPAP


In [6]:
source.出院30日[0]
source.die(30)[0]

TypeError: 'Series' object is not callable